In [161]:
workspace()
using Intervalos

#[1]
#### ¿Cuál sería una definición matemática razonable de un intervalo multi-dimensional?

Podríamos hacer un arreglo de intervalos, de modo que si tenemos `N` dimensiones entonces trabajemos con un vector de `N` entradas donde cada entrada es un objeto tipo `Intervalo`

#[2]
#### ¿Cómo podríamos expresar esto en Julia? Da dos posibilidades.

##### [Pista: Checa typealias]


MÉTODO #1

Una opción es definir un nuevo tipo que sea un arreglo de intervalos

In [144]:
type IntervalDim{N}
    a::Vector{Intervalo} #Array{Intervalo,1}
    IntervalDim(a) = ( length(a)==int(N) ? new(a) : error("Error de dimensión"))
end


Base.show(io::IO, a::IntervalDim) = print(io::IO, "IntervalDim{$(length(a.a))}
$(a.a)");

In [145]:
IntervalDim{3}([Intervalo(1,2),Intervalo(1,1)])

LoadError: Error de dimensión
while loading In[145], in expression starting on line 1

In [146]:
IntervalDim{2}([Intervalo(1,2),Intervalo(1,1)])

IntervalDim{2}
[[1e+00,
2e+00],[1e+00,
1e+00]]

Como vamos a trabajar normalmente con $\mathbb{R}^2$ voy a definir uno por 'default' de dimensión dos.

In [147]:
IntervalDim(a::Array{Intervalo,1})=IntervalDim{2}(a::Array{Intervalo,1})

IntervalDim{N} (constructor with 1 method)

In [148]:
vect1=IntervalDim([Intervalo(1,2),Intervalo(1,1)])

IntervalDim{2}
[[1e+00,
2e+00],[1e+00,
1e+00]]

# 
MÉTODO #2

Otra opción es utilizar el hint (typealias):

In [149]:
typealias IntervalD{} Vector{Intervalo}

Array{Intervalo,1}

In [150]:
typeof([Intervalo(0,1),Intervalo(2,3)])==IntervalD{}

true

De modo que tenemos que definir los arreglos de intervalos de la manera usual pero podemos hacer referencia a su tipo utilizando `IntervalD`.

Por ejemplo:

In [151]:
function algo(c::IntervalD,b::Intervalo)
    c[1]=b
    c
end

algo([Intervalo(0,1),Intervalo(2,3)]  ,  Intervalo(5,5))

2-element Array{Intervalo,1}:
 [5e+00,
5e+00]
 [2e+00,
3e+00]

También puedo definir una función que me genere elementos de tamaño `n` de ese tipo:

In [152]:
intervalD(n::Int64) = Array(Intervalo,n)
intervalD()=intervalD(2)

asdf=intervalD() #este es un arreglo 'vacío' de dos elementos
if typeof(asdf)==IntervalD
    println(asdf)
end

[#undef,#undef]


#[3]
#### ¿Cuáles operaciones matemáticas necesitaremos definir sobre los intervalos multi-dimensionales? Impleméntalos en un módulo. ¿Cuál de las dos definiciones resulta más útil en ese respecto?

MÉTODO #1

Pues si utilizamos el primero tenemos que definir TODAS las operaciones, aunque la definición es fácil ya que solo debemos hacer:

In [153]:
function definidorDim!(f::Function)
    f(x::IntervalDim,y::IntervalDim)=f(x.a,y.a)
end

definidorDim! (generic function with 1 method)

In [154]:
definidorDim!(+)
definidorDim!(-)
definidorDim!(.*)
definidorDim!(./)
definidorDim!(⋅)

dot (generic function with 9 methods)

In [155]:
vect1=IntervalDim([Intervalo(1,2),Intervalo(1,1)])
vect2=IntervalDim([Intervalo(0,1),Intervalo(2,3)])

#println(vect1+vect2)
#println(vect2-vect1)
#println(vect1.*vect2)
#println(vect2./vect1)
println(vect1⋅vect2)

[2e+00,
5e+00]


# 
MÉTODO #2

No tenemos que definir operaciones ya que ya estan definidas las operaciones entre vectores

In [156]:
vect1=[Intervalo(1,2),Intervalo(1,1)]
vect2=[Intervalo(0,1),Intervalo(2,3)]

#println(vect1+vect2)
#println(vect2-vect1)
#println(vect1.*vect2)
#println(vect2./vect1)
println(vect1⋅vect2)

[2e+00,
5e+00]


#[4]
#### Considera la función $f:\mathbb{R}^2→\mathbb{R}^2$, dada por $f(x,y)=\mathsf{M}⋅\mathbf{x}$, con $\mathsf{M} =\begin{pmatrix}
2 & 1 \\
1 & 1
\end{pmatrix}.$

#### (i) Encuentra a mano la imagen del cuadrado unitario. Encuentra el intervalo multi-dimensional más pequeño que contiene este conjunto.

In [163]:
using PyPlot